## HotChili Analytics Trading Notebook Template
#### Configure by setting ALGO_NAME in cell below.
#### Run varius options (Backtest, Ingest, Live) by uncommenting one cell 

In [ ]:
%matplotlib inline
%load_ext zipline
# %reload_ext zipline # Uncomment and use this when already loaded zipline extension for magic cell usage.
hca_root_path = !echo $HCA_ROOT
hca_root_path =hca_root_path[0]
print("hca_root_path={}".format(hca_root_path))

In [ ]:
import pandas as pd
#pd.set_option("max_colwidth", 400)
#pd.set_option("display.max_rows", 100000)
#pd.set_option("display.max_columns", 1000)
#pd.set_option('precision', 2)
#pd.options.display.float_format = '{:20,.2f}'.format

In [ ]:
# Construct Algo Strategy Path Names.
# Assumes that the Strategy is in a directory with the same name as the strategy
#Assumes the strategy is located in the hca-resources directory, which is located at: 

#Supply the Strategy name here
ALGO_NAME          = "HCA_AllWeatherOptimizeVolatility" #<--- Supply name here

HCA_RESOURCES_PATH = hca_root_path + "/hca-resources/" 
ALGO_PATH          = HCA_RESOURCES_PATH + ALGO_NAME + "/" 

ALGO_BT            = ALGO_PATH + ALGO_NAME + ".py"
ALGO_BT_OUT        = ALGO_PATH + ALGO_NAME + ".pkl"

ALGO_LIVE          = ALGO_PATH + ALGO_NAME + "_Live" + ".py"

print("ALGO_NAME={}\nHCA_RESOURCES_PATH={}\nALGO_PATH={}\nALGO_BT={}\nALGO_BT_OUT={}\nALGO_LIVE={}".format(ALGO_NAME,HCA_RESOURCES_PATH,ALGO_PATH,ALGO_BT,ALGO_BT_OUT, ALGO_LIVE))
print("\nALGO_NAME Directory Contents\n")
!ls $ALGO_PATH

## Zipline Backtest: 
    Method: Jupyter Magic Cell (%%) 
    Execution of zipline code in cell containing command line command
    Uncomment first line and hit (Shift-Enter) inside the cell to run simulation backtest.

In [ ]:
# %%zipline --start=2018-1-1 --end=2020-8-10 -b sharadar-eqfd -o $ALGO_BT_OUT

# Source: adapted from various algos on quantopian
# HCA Conversion Date: 09-05-2020
# Conversion Author: Anthony Garner


import matplotlib.pyplot as plt
import numpy as np
import math


from zipline.api import order, cancel_order, get_open_orders, symbol, symbols, date_rules, time_rules, order_target_percent, record, schedule_function, get_datetime
from trading_calendars import get_calendar

def initialize(context):
    schedule_function(func=trade, date_rule=date_rules.every_day(),
                      time_rule=time_rules.market_open(),half_days=True)
    context.asserts = symbols('SPY','IEF')

    context.rebalance_date = 0
    context.fired = False
    context.rebalance_inteval = 'M'#'Q', #'D', #'M' #'Q' #'Y'

    context.asserts_position = [0.5, 0.5]
    context.volatility_policy = True
    #unused if volatility_policy is false
    context.volatility_days = 252
    context.volatility_price_history = 66
    #set at less than 1 to ensure no leverage
    context.leverage_buffer=0.90
    
def handle_data(context, data):
    record(SPY=data[symbol('SPY')].price)

def is_new_day(context, now):
    return ( (now.year > context.rebalance_date.year) or (now.month > context.rebalance_date.month) or((now.month == context.rebalance_date.month) and (now.day > context.rebalance_date.day)))             
    
def is_new_month(context, now):
    return ((now.year > context.rebalance_date.year) or ((now.year == context.rebalance_date.year) and (now.month > context.rebalance_date.month)))

def is_new_quarter(context, now):
    return ((now.year > context.rebalance_date.year) or ((now.year == context.rebalance_date.year) and (now.month == context.rebalance_date.month + 3)))
    
def is_new_year(context, now):
    return (now.year > context.rebalance_date.year)

def need_rebalance(context, now):
    return ((context.rebalance_inteval == 'Y' and is_new_year(context, now))or 
           (context.rebalance_inteval == 'Q' and is_new_quarter(context, now)) or 
           (context.rebalance_inteval == 'M' and is_new_month(context, now)) or 
           (context.rebalance_inteval == 'D' and is_new_day(context, now)))


    # Compute historical volatility  
def compute_volatility(price_history, days):  
    # Compute daily returns  
    daily_returns = price_history.pct_change().dropna().values  
    # Compute daily volatility  
    historical_vol_daily = np.std(daily_returns,axis=0)  
    # Convert daily volatility to annual volatility, assuming 252 trading days  
    historical_vol_annually = historical_vol_daily*math.sqrt(days)  
    # Return estimate of annual volatility  
    return historical_vol_annually

def compute_asserts_volatility(context, data):
    price_history = data.history(context.asserts, "price", context.volatility_price_history, "1d")
    vol = 1.0/(compute_volatility(price_history, context.volatility_days))
    #print("vol: " + str(vol))
    sum = np.sum(vol)
    context.asserts_position = vol / sum
    #print("asserts_position: " + str(context.asserts_position))

def init_portfolio(context, data):
    if context.volatility_policy:
        compute_asserts_volatility(context, data)
    for i in range(0, len(context.asserts)):
        #print("rebalance " + context.asserts[i].symbol + " to:" + str(context.asserts_position[i]*100) + "%")
        order_target_percent(context.asserts[i], context.asserts_position[i]* context.leverage_buffer)    
        
def rebalance(context, data):
    if context.volatility_policy:
        compute_asserts_volatility(context, data)
    for i in range(0, len(context.asserts)):
        if data.can_trade(context.asserts[i]):
            #print("rebalance " + context.asserts[i].symbol + " to:" + str(context.asserts_position[i]*100) + "%")
            order_target_percent(context.asserts[i], context.asserts_position[i]* context.leverage_buffer)    
def trade(context, data):
    if not context.fired:
        context.rebalance_date = get_datetime()
        #print("build portfolio at " + str(context.rebalance_date))
        init_portfolio(context, data)
        context.fired = True
    else:
        now = get_datetime()
        if (need_rebalance(context, now)):
            #print("new rebalance arrivid:" + str(now))
            context.rebalance_date = now
            rebalance(context, data)

def analyze(context, perf):
    ax1 = plt.subplot(211)
    perf.portfolio_value.plot(ax=ax1)
    ax2 = plt.subplot(212, sharex=ax1)
    perf.SPY.plot(ax=ax2)
    plt.gcf().set_size_inches(18, 8)
    plt.show()

## Ingest Sharadar Funds Assets for today.

In [ ]:
# Ingest Sharadar Funds Assets for today, if needed.
# Only need to ingest Funds for this algo, and this takes less processing time and system memory than ingesting
#  all of Sharadar Equities plus Funds bundle (sharadar-eqfd)
# !zipline bundles # Finds all bundles
# !date
#!zipline ingest -b sharadar-funds

## Zipline Backtest: 
    Method: Command Line 
    Execution of zipline code, located in a file, using below command line execution using magic (!) invocation.
    This line can also be run in a terminal by Copying everything past the ! and pasting (Shift-Insert) it into the target terminal

In [ ]:
#!zipline run -f $ALGO_BT  --start=2018-1-1 --end=2020-10-02 -b sharadar-funds -o $ALGO_BT_OUT

## Zipline Live: 
    Method: Command Line 
    Execution of zipline code using below command line execution using magic (!) invocation.
####    **Note:** IB-TWS or IB-Gateway must be running, with broker-acct and broker-uri port being correct to live trade.

## Run Live via Command Line
### Uncomment '!zipline run ...' command below to run zipline live on IB-TWS/IB-Gateway 

In [ ]:
date_str=!date +%Y-%m-%d
TODAY = date_str[0]
print("TODAY={}".format(TODAY))

In [ ]:
ALGO_STATE = ALGO_PATH+  "strategy.state" 
ALGO_RTB   = ALGO_PATH + "realtime-bars/"

#Uncomment line below(remove prefix #'s') and run cell( Shift-Enter) to execute Live run.
# !zipline  run -s $TODAY -f $ALGO_LIVE  --bundle sharadar-funds --broker ib --broker-uri 127.0.0.1:7497:1301 --broker-acct DU1568488 --data-frequency daily  --state-file  $ALGO_STATE --realtime-bar-target $ALGO_RTB 